In [16]:
%load_ext autoreload
%autoreload 2
import pprint, json, math, os, sys
# dir_path = "C:\\Users\\rando\\OneDrive\\Documents\\mywork-repo"
# fund_path = r"C:\Users\rando\OneDrive\Documents\PDFDrive\Mar25"
# dry_path = r"C:\Users\rando\OneDrive\Documents\mywork-repo\notebook\DryRun.pdf"

dir_path = "C:\\Users\\Kaustubh.keny\\Projects\\office-work\\mywork-repo"
fund_path = "C:\\Users\\Kaustubh.keny\\Projects\\Mar 25"
dry_path = r"C:\Users\Kaustubh.keny\Projects\office-work\mywork-repo\notebook\DryRun.pdf"
sys.path.append(os.path.abspath(dir_path))
from app.config_loader import load_config_once
conf = load_config_once()


import fitz, pdfplumber, ocrmypdf
import pandas as pd
import numpy as np
from collections import defaultdict

from app.utils import Helper
from app.parse_regex import *

dry_path = r'DryRun.pdf'
fin_path = r'\data\input\financial_indices.xlsx'
mutual_fund = Helper.get_fund_paths(fund_path)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Config already loaded. Skipping re-initialization.


In [19]:
def extract_clipped_data(input:str, pages:list, bboxes:list):
        
        document = fitz.open(input)
        final_list = []
    
        for pgn in pages:
            page = document[pgn]
            
            all_blocks = [] #store every data from bboxes
            
            for bbox in bboxes:
                blocks, seen_blocks = [], set()  #store unique blocks based on content and bbox
                
                page_blocks = page.get_text('dict', clip=bbox)['blocks']
                for block in page_blocks:
                    if block['type'] == 0 and 'lines' in block: #type 0 means text block
                        #hash_key
                        block_key = (tuple(block['bbox']), tuple(tuple(line['spans'][0]['text'] for line in block['lines'])))
                        if block_key not in seen_blocks:
                            seen_blocks.add(block_key)
                            blocks.append(block)

                sorted_blocks = sorted(blocks, key=lambda x: (x['bbox'][1], x['bbox'][0]))
                all_blocks.append(sorted_blocks)

            final_list.append({
                "pgn": pgn,
                "block": all_blocks #will be list[list,list,..]
            })

        document.close()
        return final_list
    
def extract_data_relative_line(path: str, line_x: float, side: str):
    doc = fitz.open(path)
    pages = doc.page_count

    final_list = []

    for pgn in range(pages):
        page = doc[pgn]

        blocks = page.get_text("dict")["blocks"]
        sorted_blocks = sorted(blocks, key=lambda x: (x["bbox"][1], x["bbox"][0]))
        extracted_blocks = []

        # Keep track of blocks to avoid duplicates
        added_blocks = set()

        for block in sorted_blocks:
            block_id = id(block)  # Unique identifier for the block

            for line in block.get("lines", []):
                for span in line.get("spans", []):
                    origin = span["origin"]
                    x0, _ = origin

                    # Check the side condition
                    if side == "left" and x0 < line_x and block_id not in added_blocks:
                        extracted_blocks.append(block)
                        added_blocks.add(block_id)  # Mark block as added
                    elif side == "right" and x0 > line_x and block_id not in added_blocks:
                        extracted_blocks.append(block)
                        added_blocks.add(block_id)  # Mark block as added

      
        final_list.append({
            "pgn": pgn,
            "blocks": extracted_blocks
        })

    doc.close()

    return final_list
  
def get_clipped_data(input:str, bboxes:list[set], *args):
    
        document = fitz.open(input)
        final_list = []
        if args:
            pages = list(args)
        else:
            pages = [i for i in document.page_count]
        
        for pgn in pages:
            page = document[pgn]

            blocks = []
            for bbox in bboxes:
                blocks.extend(page.get_text('dict', clip = bbox)['blocks']) #get all blocks
            
            filtered_blocks = [block for block in blocks if block['type']== 0 and 'lines' in block]
            # sorted_blocks = sorted(filtered_blocks, key= lambda x: (x['bbox'][1], x['bbox'][0]))
            
             # Extract text from sorted blocks
            extracted_text = []
            for block in filtered_blocks:
                block_text = []
                for line in block['lines']:
                    line_text = " ".join(span['text'] for span in line['spans'])
                    block_text.append(line_text)
                extracted_text.append("\n".join(block_text))
            
            final_list.append({
            "pgn": pgn,
            "block": filtered_blocks,
            "text": extracted_text
            })
            
            
        document.close()
        return final_list
    
def extract_clipped_text_all_pages(pdf_path, clip_coords):
    results = {}
    doc = fitz.open(pdf_path)
    clip_rect = fitz.Rect(*clip_coords)
    try:
        for page_number, page in enumerate(doc):
            text = page.get_text("text", clip=clip_rect).strip()
            results[page_number] = text
    finally:
        doc.close()
    return results

In [26]:

sample_path = mutual_fund["Edelweiss Mutual Fund"]
rp = r"\pdfs\unifi1747199772321.pdf"
rp = r"\pdfs\unifi1747373013796.pdf"
rp = r"\pdfs\unifi1747199156275.pdf"
dsp1 = "\\pdfs\\May SID\\DSP Mutual Fund\\8_17160_May-2025_1748246407_SID.pdf"
tata1 = r"\pdfs\May SID\Tata Mutual Fund\38_17082_May-2025_1746423571_SID.pdf"
nip1 = "\\pdfs\\May SID\\Nippon India Mutual Fund\\33_17142_May-2025_1747631690_SID.pdf"
sbi1 = "\\pdfs\\May SID\\SBI Mutual Fund\\35_17126_May-2025_1747282059_SID.pdf",
samc1 = "\\pdfs\\May SID\\Samco Mutual Fund\\58_17166_May-2025_1748246684_SID.pdf"
sample_path = dir_path + samc1

In [ ]:
lines = [
    ((260, 0), (260, 812)),# Vertical line
    ((0, 450), (812, 450))
]
pages = [12, 14,16]
bboxes = [[0, 120, 180, 812],[180, 85, 360, 812]] #[(0, 85, 180, 812),(180, 85, 360, 812),(0,100,270,812),(0,100,350,812)]
pages = [i for i in range(1,110)]
Helper.draw_lines_on_pdf(sample_path, lines, bboxes, pages, dry_path)

Modified PDF saved to: DryRun.pdf


In [ ]:

clip_coords = (484, 50,600, 100)  # (x0, y0, x1, y1)

clipped_texts = extract_clipped_text_all_pages(sample_path, clip_coords)

for page_num, text in clipped_texts.items():
    print(f"Page {page_num}:\n{text}\n{'-'*40}")


In [ ]:
# def _detect_table_start_by_keywords(df: pd.DataFrame, keywords: list):
#     normalized_keywords = [re.sub(r"\s+", " ", kw.strip().lower()) for kw in keywords]

#     for i in range(df.shape[0]):
#         for j in range(df.shape[1]):
#             cell = str(df.iat[i, j]).strip().lower()
#             cell = re.sub(r"\s+", " ", cell)
#             if any(kw in cell for kw in normalized_keywords):
#                 return i, j
#     return 0, 0


In [ ]:
data = Helper.get_all_pdf_data(sample_path)s

In [ ]:
def get_proper_fund_names(path: str):
    pattern = "(HDFC.*?(?:FUND|Fund|ETF|FO?o?F)\\s*(?:of Funds?|.+?Plan|Fund of Funds?|Fund)?)"
    title = {}   
    with fitz.open(path) as doc:
        for pgn, page in enumerate(doc):
            text = " ".join(page.get_text("text", clip=(0, 0, 400, 60)).split("\n"))
            text = re.sub("[^A-Za-z0-9\\s\\-\\(\\).,]+|\u2028", "", text).strip()
            print(text)
            if matches := re.findall(pattern, text, re.DOTALL):
                title[pgn] = " ".join([_ for _ in matches[0].strip().split(" ") if _])
                print(pgn,matches[0])
    return title
title = get_proper_fund_names(sample_path)

In [ ]:
import os ,json 
import pandas as pd

full_name = set()
split_name = set()
MANAGER_REGEX = FundRegex().MANAGER_STOP_WORDS
for a,b,files in os.walk(r"C:\Users\Kaustubh.keny\Projects\office-work\mywork-repo\sql_learn\json\MAR25DATA"):
    for paths in files:
        sample_path = os.path.join(os.getcwd(),"..","sql_learn","json","MAR25DATA",paths)
        # print(sample_path)
        try:
            with open(sample_path, "r", encoding="utf-8") as file:
                data = json.load(file)
                for k,scheme in data.items():
                    # print(k)
                    if "fund_manager" in scheme:
                        for entry in scheme['fund_manager']:
                            # print(entry['name'])
                            name = entry['name']
                            for regex_ in MANAGER_REGEX:
                                name = re.sub(f"\\b{regex_}\\b|[^A-Za-z\\s0-9]+","",name, re.IGNORECASE)
                                name = re.sub(r"\s+"," ",name)
                            full_name.add(name)
                            printthis = name if name.strip() else "EMPTY"
                            # print(f"<<{printthis}>>")
                            if printthis == "EMPTY":
                                print(k,printthis,entry['name'])
                            # split_name.add(name.split(' '))
                    # print(scheme.keys())
        except Exception as e:
            print(f"NEVER MIND {e}")
    # print(files)
        